# Semianr 5 - Applied Quantitative Logistics

In [29]:
import math
import random
import matplotlib.pyplot as plt
import numpy as np

## Hub Location Allocation

In [31]:
N = 40    # Number of customers
M = 20    # Number of centers

# Generate the x coordinate of customer
xc = list(np.random.randint(0, 100, N))
print(xc)

[94, 60, 62, 3, 23, 43, 27, 62, 97, 75, 54, 56, 58, 0, 62, 28, 27, 86, 7, 52, 65, 0, 32, 15, 3, 61, 16, 65, 63, 43, 67, 53, 98, 68, 34, 32, 26, 18, 33, 66]


In [32]:
# Generate the y coordinate of customer
yc = list(np.random.randint(0, 100, N))
print(yc)

[67, 33, 11, 2, 48, 88, 17, 71, 38, 78, 37, 72, 58, 12, 54, 5, 24, 76, 38, 61, 82, 65, 66, 55, 17, 79, 72, 14, 45, 30, 36, 97, 15, 52, 30, 66, 13, 42, 77, 12]


In [34]:
# Generate demand for each customer
d = list(np.random.randint(5, 50, N))
print(d)

[13, 44, 12, 30, 5, 44, 18, 29, 20, 6, 35, 16, 17, 31, 32, 9, 37, 30, 8, 21, 31, 17, 47, 11, 6, 14, 40, 48, 31, 16, 32, 14, 44, 42, 6, 26, 20, 43, 14, 20]


In [37]:
# Generate the x coordinate of centers
xs = list(np.random.randint(0, 99, M))
print(xs)

[63, 91, 0, 54, 39, 55, 15, 50, 21, 92, 12, 81, 41, 5, 62, 75, 62, 20, 77, 57]


In [38]:
# Generate the y coordinate of centers
ys = list(np.random.randint(0, 99, M))
print(ys)

[37, 25, 37, 12, 43, 57, 98, 2, 85, 87, 0, 75, 30, 30, 85, 14, 65, 78, 95, 36]


In [ ]:
def hubLocation():
    
    # Customer information
    xc = [94, 60, 62, 3, 23, 43, 27, 62, 97, 75, 54,
          56, 58, 0, 62, 28, 27, 86, 7, 52, 65, 0, 32,
          15, 3, 61, 16, 65, 63, 43, 67, 53, 98, 68,
          34, 32, 26, 18, 33, 66]
    
    yc = [67, 33, 11, 2, 48, 88, 17, 71, 38, 78,
          37, 72, 58, 12, 54, 5, 24, 76, 38, 61,
          82, 65, 66, 55, 17, 79, 72, 14, 45, 30,
          36, 97, 15, 52, 30, 66, 13, 42, 77, 12]
    
    d = [13, 44, 12, 30, 5, 44, 18, 29, 20, 6, 35,
         16, 17, 31, 32, 9, 37, 30, 8, 21, 31, 17,
         47, 11, 6, 14, 40, 48, 31, 16, 32, 14, 44,
         42, 6, 26, 20, 43, 14, 20]
    
    N = len(xc)
    
    # Service center information
    xs = [63, 91, 0, 54, 39, 55, 15, 50, 21, 92,
          12, 81, 41, 5, 62, 75, 62, 20, 77, 57]
    
    ys = [37, 25, 37, 12, 43, 57, 98, 2, 85, 87,
          0, 75, 30, 30, 85, 14, 65, 78, 95, 36]
    
    M = len(xs)
    
    # calculate the distance
    D = np.zeros([N, M])
    
    for i in range(N):
        for j in range(M):
            D[i][j] = math.sqrt(abs((xc[i]-xs[j])**2 + (yc[i]-y[j])**2))
    
    model = {
        "N" : N,
        "M" : M,
        "xc" : xc,
        "yc" : yc,
        "d" : d,
        "xs" : xs,
        "ys" : ys,
        "D" : D
    }
    return model

In [ ]:
model = hubLocation()
model['D'].shape

In [ ]:
model['ys'][4]

### Create Random Solution

In [55]:
def createRandomSolution(model):
    M = model['M']
    f = list(np.random.randint(0, 2, M))
    
    return f

In [58]:
createRandomSolution(model)

[1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0]

### Cost Function

In [ ]:
def myCost(f, model):
    
    global NFE
    
    if pd.isna(NFE):
        NFE = 0

    NFE += 1
    
    # If no center is activated
    # The cost of the system is inf
    if (np.all(np.array(f) == 0)):
        z = math.inf
        return z

    N = model['N']
    M = model['M']
    D = model['D']
    
    D_min = np.zeros(N)
    
    for i in range(N):
        D_temp = []
        for j in range(M):
            if f[j] == 1:
                D_temp.append(D[i][j])
                
        D_min[i] = min(D_temp)
        
    z = sum(np.array(model['d']) * np.array(D_min))
    
    return z

### Sorting Population

### Roullete Wheel Selection

### Crossover

### Mutation

### GA Algorithm

### Results

In [ ]:
# Plot the result
plt.plot(bestCosts, linewidth = 3)
plt.xlabel('NFE')
plt.ylabel('Costs')

# HW6 - Applied Quantitative Logistics

Solve TSP problem with continues GA algorithm. The solution should be generated in continues values. Use all regular Crossover and Mutation. This type of problem solving has no interfere with permutaion problems.

Hint: Just before calculating the cost, you need to convert the solution into the permutation format. For example:

- Solution in continues format is: [0.111, 0,854, 0,234, 0.542, 0.01]
- The permutation format is: [2, 5, 3, 4, 1]

Instruction for submission:

- Please submit your solutions in (.ipynb) format to my email (msohrabi@hse.ru).

- Deadline: **April 28, 2023, 11:59 pm.**

- The subject of the email and file name: **[HW6_AQL]-YOUR_NAME**